# About
Estimation of distribution of dynamical and diffusional parameters of Ito's 
processes. This approach don't imply any functional representation of parameters
as in most of the know models.

## Brief overview

$S(t) = S_t$ is called an Ito process if it satisfies next equality:
$$ 
dS_t = a(t, S_t) \cdot dt + b(t, S_t) \cdot dW 
$$ (Ito_proc)


We can create estimate $a(t, S_t)$ and $b(t, S_t)$ in two ways:
1. By their distribution
2. Point-to-point


## 1. Estimation of Process distribution

Let $S(t_i) = S_i, \quad a(t_i, S_i) = a_i, \quad b(t_i, S_i) = b_i$ and
$ i \in \{1, ..., T\}$. Then
$$
\Delta S = S_i - S_{i-1} \approx a(t_i, S_i) \Delta t + b(t_i, S_i) \Delta W 
$$ (my_label)

$$
\Rightarrow
\mathbb{P}(\Delta S < x) \approx \mathbb{P}(\Delta W < \frac{x - a_i \Delta t}{b_i}),
\quad \text{where } \Delta W \sim \mathcal{N}(0,\Delta t)\,
$$

Here $\Delta t$ denotes time discretization step.

### Law of total probability

Let denote $\frac{x - a_i \Delta t}{b_i} = \xi(x, a_i, b_i, t)= \xi$. It is a r.v. 
Therefore $\mathbb{P}(\Delta W < \xi) \equiv \Phi(\frac{\xi - 0}{\Delta t})$

Keeping in mind that $a_i$ and $b_i$ are r.v. Without loss of generality 
let's suppose that $\forall t: \; a_i \in A \subset \mathbb{R} \wedge b_i \in B \subset \mathbb{R^+j}$.

Let $f_{ab}$ denote joint distribution density for r.v. $a_i$ and $b_i$.
Then
$$
\mathbb{P}(\Delta W < \xi) = 
\int_{A}\int_{B} \, \mathbb{P}(\Delta W < \xi \: | \:a_i=y \wedge b_i=z) \cdot f_{ab}(y, z)\, dy \, dz =
\newline
\int_{A}\int_{B} \, \Phi(\frac{x - y \Delta t}{z \Delta t}) \cdot f_{ab}(y, z)\, dy \, dz =
\{\;f_{ab}(y_k, z_k) = p_k \;\} \approx
\sum_{k=1}^{K}{p_k \cdot \Phi \left( \frac{x - y_k \Delta t}{z_k \Delta t} \right)} 
$$

$$
\text{Small note: as a result we can get that} \quad
\mathbb{P}(dW < \xi) = 
\mathbb{E}_{a,b}(\Phi(\frac{x - a \cdot dt}{b \cdot dt})) 
$$

Therefore we can approximate initial process distribution as:

$$
\mathbb{P}(\Delta S < x) \approx
\sum_{k=1}^{K}{p_k \cdot \Phi \left( \frac{x - y_k \Delta t}{z_k \Delta t} \right)} 
$$

## Aspects to consider
1. How to choose limits for $A$ and $B$? Is it bad to consider them constant for
each $t$?
2. How to choose points $\{(y_k, z_k)\}_{k=1}^{K}$? Obviously the best approach 
is to select extremum of the joint distribution function $f_{ab}$.
3. Are $a$ and $b$ dependant r.v.? $\Leftrightarrow$ Can we assume $f_{ab} = f_a \cdot f_b$?

Answer to question 3 define dimensionality of optimization task

In [ ]:
from finito.simulator import generateGeneralWiener
from finito.gauss import GaussianMixture
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

# Generate process
dummyS = generateGeneralWiener(
    a=3, b=0.4, dt=np.timedelta64(1, "ms"), T=np.timedelta64(10, "s")
)
print("Shape of data:", dummyS.shape)

#### 1.1. EM-algorithm

In [ ]:
# Take differences
deltaS = np.diff(dummyS)

# Take a look on result
f, ax = plt.subplots(3, 1, figsize=(10, 10))
ax[0].plot(dummyS)
ax[0].set_title("Original process")
ax[1].plot(deltaS)
ax[1].set_title("Differences")
sns.histplot(deltaS, kde=True, ax=ax[2])
ax[2].set_title("Distribution of differences")

plt.show()

In [ ]:
from sklearn.mixture import GaussianMixture as GM

gmm = GM(
    n_components=1,
)

gmm.fit(deltaS.reshape(-1, 1))

print(
    "Model parameters after fitting:",
    f"Means: {gmm.means_[0]}",
    f"Covariances: {gmm.covariances_.flatten()}",
    sep="\n",
)
__time_delta = np.timedelta64(1, "s") / np.timedelta64(1, "ms")
print(
    "As we can see it differs from initial a and b parameters on time delta:",
    __time_delta,
)
print(
    f"Means: {gmm.means_ * __time_delta}",
    f"Covariances: {gmm.covariances_ * __time_delta}",
    f"Standard deviation: {np.sqrt(gmm.covariances_ * __time_delta)}",
    sep="\n",
)

### 1.2. Minimization problem on distribution

Discrete approximation $F_{\Delta S, T}(x)$ of the continuous distribution 
$F_{dS}(x)$ can be estimated in two ways: theoretical and empirical. 

On one hand:
$$
F^{(theor)}_{\Delta S, T}(x) = \mathbb{P}(\Delta S < x) \approx 
\sum_{k=1}^{K}{p_k \cdot \Phi \left( \frac{x - y_k \Delta t}{z_k \Delta t} \right)} 
\xrightarrow{K \rightarrow \infty , \,\Delta t \rightarrow 0} F_{dS}
$$

But on the other we can estimate empirical distribution of $\Delta S$ as:

$$
F^{(emp)}_{\Delta S, T}(x) = 
\frac{1}{T} \cdot \sum_{j=1}^{T} \mathbb{I} \left(\Delta S < x\right)
\xrightarrow{\Delta t \rightarrow 0} F_{dS}
$$

For evaluation of parameters $V_k = \{y_k, z_k, p_k\}$ we can set an 
minimization problem on distance (in some metric $\rho(f,g)$) between
theoretical and empirical distributions:

$$
$$

$$ 
\begin{equation}
\left\{ \begin{aligned} 

    \min_{\{V_k\}_{k=1}^{K}} \rho \left(F^{(emp)}_{\Delta S, T}, F^{(theor)}_{\Delta S, T}\right) \\
    p_k \geq 0 \; \wedge \; \sum_{k=1}^{K}p_k = 1 \\
    0 \leq z_k \in B \subset \mathbb{R^+} \\
    y_k \in A \subset \mathbb{R}



\end{aligned} \right.
\end{equation}
$$


#### 1.2.1. Empirical distribution implementation

In [ ]:
# TODO: Optimize cycle. Currently O(M*N), can be O(N), where M is len(x), N is len(data)
from typing import Iterable


def EmpiricalCDF(data: Iterable, x: Iterable | None = None):
    """Estimate cumulative distribution function.

    Args:
        data (Iterable): Samples of the random variable
        x (Iterable | None): Specific points to calculate distribution in

    Returns:
        _type_: _description_
    """
    # Use all possible information for estimation
    if x is None:
        x = data
    res = []
    nSamples = len(data)
    for xi in x:
        res.append(np.sum(data <= xi) / nSamples)
    res.sort()
    return np.array(res)

In [ ]:
# Represent empirical distribution function correspondence to theoretical
# distributions for a basic distributions
fig, ax = plt.subplots(1, 2, figsize=(10, 5))

# Binomial
ax[0].set_title("Binomial distribution")
ax[1].set_title("Normal distribution")
binomRV = stats.binom(n=10, p=0.5)
X = np.arange(-3, 13)
sns.pointplot(x=X, y=binomRV.cdf(X), ax=ax[0], color="blue")
nSamples = 240
binomSamples = binomRV.rvs(size=nSamples)
X = np.arange(min(binomSamples), max(binomSamples))
Xm = X[::1]  # To select discretization step
sns.pointplot(x=Xm, y=EmpiricalCDF(binomSamples, Xm), ax=ax[0], color="orange")
# Standard normal
nSamples = 200
normRV = stats.norm(loc=0, scale=1)
X = np.linspace(-3, 3, nSamples)
sns.pointplot(x=X, y=normRV.cdf(X), ax=ax[1], color="blue")
normSamples = normRV.rvs(size=nSamples)
Xm = X[::10]
sns.pointplot(x=Xm, y=EmpiricalCDF(normSamples, Xm), ax=ax[1], color="orange")
ax[1].tick_params(axis="x", which="both", labelsize=0)

#### 1.2.2 Gaussian mixture model implementation

In [ ]:
a = [-1, 3, 5]
b = [0.3, 0.5, 0.2]

gm = GaussianMixture(a=a, b=b)
r = gm.get_params()
np.stack(r)
gm.sample(10)

In [ ]:
a = [-1, 3, 5]
b = [0.3, 0.5, 0.2]

gm1 = GaussianMixture(a=a, b=b)
x_grid = np.linspace(-3, 7, 100)
cdf_values = gm1.cdf(x_grid)
print(
    "Plots are different time to time due to random weights generation.",
    f"\nCurrent weights: {gm1._weights.flatten()}",
)

pdf_values = gm1.pdf(x_grid)

plt.figure(figsize=(8, 5))
plt.plot(x_grid, pdf_values, label="GMM PDF", color="blue")
plt.title("PDF of Gaussian Mixture Model")
plt.xlabel("x")
plt.ylabel("PDF")
plt.legend()
plt.grid()
plt.show()

sns.histplot(gm1.sample(size=1000), kde=True)

In [ ]:
from sklearn.metrics import mean_absolute_error

samples = gm1.sample(500)
x = sorted(samples)
plt.figure(figsize=(8, 5))
origCDF = gm1.cdf(x)
plt.plot(x, origCDF, label="Original CDF", color="black")
empirCDF = EmpiricalCDF(samples)
plt.plot(x, empirCDF, label="Empirical CDF", color="green")
plt.title("CDF of Gaussian Mixture Model")
plt.xlabel("Random variable values")
plt.ylabel("Probability")
plt.legend()
plt.grid()
plt.show()

print(
    f"Number of samples: {len(samples)}",
    f"Mean absolute error: {mean_absolute_error(origCDF, empirCDF) * 100:.2f}%",
    sep="\n",
)

In [ ]:
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint, OptimizeResult


def distance(params, x, dist=np.abs):
    mu, sigma, weight = params.reshape(3, 3, -1)
    F_n = EmpiricalCDF(x)
    gaus = stats.norm(loc=mu, scale=sigma)
    F_theory = np.sum(weight * gaus.cdf(sorted(x)), axis=0)
    # Evaluate KS statistic: max |F_n(t) - F(t; mu, sigma)|
    ks_vals = dist(F_n - F_theory)
    return float(np.max(ks_vals))


def distF(x):
    return lambda params: distance(params, x)


bounds = [
    (-5, 5),
    (-5, 5),
    (-5, 5),
    (1e-2, 1),
    (1e-2, 1),
    (1e-2, 1),
    (0, 1),
    (0, 1),
    (0, 1),
]
initial_guess = [
    np.random.uniform(*bounds[0], 3),
    np.random.uniform(*bounds[3], 3),
    np.random.dirichlet(np.ones(3)),
]
initial_guess = np.array(initial_guess).flatten()
A = np.array(
    [
        [0, 0, 0, 0, 0, 0, 1, 1, 1]  # Constraint on probs sum
    ]
)
lb = np.array([1])  # Lower bounds for the constraints
ub = np.array([1])  # Lower bounds for the constraints
linear_constraint = LinearConstraint(A, lb, ub)


# res = dual_annealing(
# res = basinhopping(

res = OptimizeResult(fun=float("inf"))
while res.fun > 0.02:
    res = minimize(
        distF(samples),
        initial_guess,
        bounds=bounds,
        constraints=[linear_constraint],
        method="SLSQP",
    )
    initial_guess = [
        np.random.uniform(*bounds[0], 3),
        np.random.uniform(*bounds[3], 3),
        np.random.dirichlet(np.ones(3)),
    ]
    initial_guess = np.array(initial_guess).flatten()
    print(res.fun)
print(res)
predictions = res.x.reshape(3, -1)
predictions = [list(map(lambda x: float(round(x, 6)), param)) for param in predictions]
print(
    "Predicted parameters",
    *predictions,
    "Original parameters",
    *np.stack(gm1.get_params()),
    sep="\n",
)

In [ ]:
gmP = GaussianMixture(*predictions)
x = sorted(samples)
fig, axes = plt.subplots(1, 2, figsize=(8 * 2, 5))
origCDF = gm1.cdf(x)
axes[0].plot(x, origCDF, label="Original CDF", color="black")
empirCDF = EmpiricalCDF(samples)
axes[0].plot(x, empirCDF, label="Empirical CDF", color="green")
axes[0].plot(x, gmP.cdf(x), label="Predicted GMM CDF", color="red")
axes[0].set_title("CDF of Gaussian Mixture Model")
axes[0].set_xlabel("Random variable values")
axes[0].set_ylabel("Probability")
axes[0].legend()
axes[0].grid()

axes[1].plot(x, gm1.pdf(x), label="Original PDF", color="black")
empirCDF = EmpiricalCDF(samples)
# axes[1].plot(x, numDeriv(empirCDF, x), label="Empirical PDF", color="green")
axes[1].plot(x, gmP.pdf(x), label="Predicted GMM PDF", color="red")
axes[1].set_title("PDF of Gaussian Mixture Model")
axes[1].set_xlabel("Random variable values")
axes[1].set_ylabel("Probability")
axes[1].legend()
axes[1].grid()
fig.show()